In [1]:
from tqdm import tqdm
from init import *
import gif
import pyvortex as vr
from mpl_toolkits.basemap import Basemap, addcyclic, cm

autotime loaded.


In [2]:
def get_basemap(ax = None, min_lat = 30):
    if not ax:
        fig, ax = facets(1, 1, width = 10, aspect = 0.6); ax = ax[0]
    m = Basemap(projection = 'spstere', lon_0 = 180, boundinglat = min_lat, \
                round = True, ax = ax)
    m.drawcoastlines(linewidth=0.35)
    m.drawcountries(linewidth=0.35)
    m.drawmeridians(np.arange(0, 360, 60), labels = [0,0,0,0])
    m.drawparallels(np.arange(20, 80, 20), labels = [1,1,0,0])
    return m


@gif.frame
def plot(time, min_lat=-20): 
    fig, axes = plt.subplots(2, 3, figsize=(15, 11))
    axes = axes.flatten()
    cax = fig.add_axes([0.265, 0.08, 0.5, 0.02])
    levels = [400, 450, 500, 550, 600, 650]
    for (level, ax) in zip(levels, axes):
        m = get_basemap(ax=ax, min_lat=min_lat)
        dic = {'level':level}
        pedge = eql.vedge.isel(time=time).sel(dic, method='nearest').values.astype('int')  
        pvv = eql.eql.isel(time=time).sel(dic, method='nearest').sel(lat=slice(-90, min_lat))
        lon = pvv.lon.values
        #pvv, lon = addcyclic(ppv.values, ppv.lon.values)
        xx, yy = np.meshgrid(pvv.lon, pvv.lat)
        xx, yy = m(xx, yy)
        h = m.contourf(xx, yy, pvv, cmap='jet', levels=np.arange(0, 91))
        m.contour(xx, yy, pvv, levels=[pedge], color='black', zorder=20, lw=10)
        text = 'Level: %s K \n Vedge: %s$^\circ$N'%(level, pedge)
        ax.set_title(text, x=0.5, y=1.01)
    cb = plt.colorbar(h, cax=cax, orientation='horizontal')
    cb.set_label(label = 'Equivalent Latitude [$^\circ$S]', size=25)
    plt.subplots_adjust(hspace=0.16, wspace=0.1)
    plt.suptitle('Date: %s'%str(eql.time.values[time]).split('T')[0])

time: 9.54 ms


In [3]:
files = sorted(glob.glob('/media/pankaj/ext2/antarctic/era5/2015-10*.nc'))
df = xr.open_mfdataset(files, concat_dim='time')
eql = vr.PolarVortex(-1*df['pv'][:, :, ::-1, :] , df['u'][:, :, ::-1, :]).get_edge(min_eql=30)

time: 1min 42s


In [5]:
min_lat = -20
frames = []
for time in tqdm(np.arange(len(files))):
    frame = plot(time)
    frames.append(frame)
    
gif.save(frames, 'antarctic.gif', duration=500)

100%|██████████| 31/31 [03:17<00:00,  6.36s/it]


time: 3min 20s
